In [ ]:

import polars as pl

# Load dataset
df = pl.read_csv("2024_fb_posts_president_scored_anon.csv")

# Deduplicate columns
seen = set()
deduped_cols = [col for col in df.columns if not (col in seen or seen.add(col))]
df = df.select([pl.col(c) for c in deduped_cols])

#  Numeric Summary 
numeric_df = df.select(pl.col(pl.Float64, pl.Int64))
numeric_summary = numeric_df.describe()
numeric_summary.write_csv("fb_posts_numeric_summary.csv")

#   Non-Numeric Summary  
non_numeric_df = df.select(pl.col(pl.Utf8))
non_numeric_rows = []
for col in non_numeric_df.columns:
    unique_count = df.select(pl.col(col).n_unique()).item()
    vc = df.select(pl.col(col).value_counts()).unnest(col)
    if vc.height > 0:
        most_common_val = vc[0, col]
        most_common_count = vc[0, "count"]
    else:
        most_common_val = None
        most_common_count = 0

    non_numeric_rows.append({
        "column": col,
        "unique_values": unique_count,
        "most_frequent_value": most_common_val,
        "frequency": most_common_count
    })
pl.DataFrame(non_numeric_rows).write_csv("fb_posts_non_numeric_summary.csv")

#   Group by Facebook_Id  
if "Facebook_Id" in df.columns and "Total Interactions" in df.columns:
    grouped_by_fbid = df.group_by("Facebook_Id").agg([
        pl.len().alias("post_count"),
        pl.col("Total Interactions").mean().alias("avg_interactions"),
        pl.col("Total Interactions").min().alias("min_interactions"),
        pl.col("Total Interactions").max().alias("max_interactions")
    ])
    grouped_by_fbid.write_csv("fb_posts_grouped_by_facebook_id.csv")

#   Group by Facebook_Id and post_id  
if all(col in df.columns for col in ["Facebook_Id", "post_id", "Total Interactions"]):
    grouped_by_fbid_post = df.group_by(["Facebook_Id", "post_id"]).agg([
        pl.len().alias("entry_count"),
        pl.col("Total Interactions").mean().alias("avg_interactions"),
        pl.col("Total Interactions").min().alias("min_interactions"),
        pl.col("Total Interactions").max().alias("max_interactions")
    ])
    grouped_by_fbid_post.write_csv("fb_posts_grouped_by_facebook_post.csv")

print("✅ Facebook posts analysis CSVs saved.")

✅ Facebook posts analysis CSVs saved.
